In [13]:
import yaml
from pathlib import Path
from CrystalTracer3D.io import CrystalReader
with open('config.yml', 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    in_img = Path(cfg['data']['path'])
    out_dir = Path(cfg['data']['segmentation'])
    seg_chan = cfg['data']['neuron']
    slice_range = cfg['data']['range']


reader = CrystalReader(in_img)
ch = reader.find_channel(seg_chan)
img = reader.read(ch)[slice_range[0]:slice_range[1]]


In [14]:
import napari

viewer = napari.Viewer(ndisplay=3)
ch = reader.find_channel(cfg['data']['crystal'])
viewer.add_image(reader.read(ch))

<Image layer 'Image' at 0x17b34bd5060>

In [17]:
viewer.layers[0].contrast_limits

[11967.714285714284, 25246.285714285714]

smoothing

In [10]:
from skimage.filters import gaussian

smoothed = gaussian(img[:350], 3)

In [16]:
import napari

viewer = napari.Viewer()
viewer.add_image(img)
napari.run()

find peaks morphologically

In [5]:
from skimage.morphology import reconstruction
import numpy as np
from tqdm import tqdm

rec = []
for i in tqdm(smoothed):
    seed = np.copy(i)
    seed[1:-1, 1:-1] = i.min()
    mask = i
    i = reconstruction(seed, mask, method='dilation')
    rec.append(i)
rec = np.stack(rec)
peaks = smoothed - rec

100%|██████████| 390/390 [05:36<00:00,  1.16it/s]


In [57]:
import napari

viewer = napari.Viewer()
viewer.add_image(peaks)
napari.run()

In [6]:
import pyclesperanto_prototype as cle

labels = cle.voronoi_otsu_labeling(peaks, spot_sigma=10, outline_sigma=1)


In [7]:
import napari

viewer = napari.Viewer()
viewer.add_image(peaks)
viewer.add_labels(labels)
napari.run()

# Test segmentation function

In [17]:
from CrystalTracer3D.segment import segment_soma

seg = segment_soma(img[:330])

  0%|          | 0/330 [00:00<?, ?it/s]

In [18]:
import napari

viewer = napari.Viewer()
viewer.add_image(img[:330])
viewer.add_labels(seg)
napari.run()

# Check Crystal

In [13]:
import nrrd

data, _ = nrrd.read(r"D:\Zuohan\neuron\crystal.nrrd")

In [74]:
viewer = napari.Viewer()
viewer.add_labels(data, opacity=.9, units=reader.resolution)
viewer.add_image(img, opacity=.7, units=reader.resolution)
napari.run()
viewer.camera.perspective = 45
viewer.dims.ndisplay = 3
viewer.camera.zoom = 0.8
viewer.camera.angles = (-5, 30, 80)

In [51]:
viewer.screenshot(path=r"D:\Zuohan\neuron\crystal_check.png", scale=2, canvas_only=True)

array([[[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       ...,

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0

# Check neuron

In [7]:
import nrrd
import napari

data, _ = nrrd.read(r"D:\Zuohan\neuron2\neuron.nrrd")

In [10]:
viewer = napari.Viewer()
viewer.add_labels(data, opacity=.9, scale=reader.resolution, blending='additive')
viewer.add_image(img, opacity=.7, scale=reader.resolution)
napari.run()
viewer.layers[0].iso_gradient_mode = 'smooth'
viewer.camera.perspective = 45
viewer.dims.ndisplay = 3
viewer.camera.zoom = 0.8
viewer.camera.angles = (-5, 30, 80)

In [12]:
viewer.layers[1].contrast_limits

[2840.0573248407645, 13763.904458598727]

In [68]:
viewer.screenshot(path=r"D:\Zuohan\neuron\neuron_check.png", scale=2, canvas_only=True)

array([[[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       ...,

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        ...,
        [  0,   0,   0, 255],
        [  0,   0,   0, 255],
        [  0,   0,   0, 255]],

       [[  0